In [ ]:
import robin_stocks as rs #the robinhood api wrapper
import pandas as pd #for creating CSV files
from datetime import timedelta, datetime #for getting current time and converting recent news timestamps
import time #for sleep timer
import csv #for creating CSV file of stock purchase list

In [ ]:
#include functions from notebooks
%run auth.ipynb
%run recent_stock_news.ipynb
%run check_volume.ipynb
%run get_trending_stocks.ipynb
%run purchase_stock.ipynb
%run tradeable_stocks.ipynb
%run cloud_CSV_storage.ipynb

In [ ]:
#authenticate robinhood account
get_header = rh_auth(username='', password='')

In [ ]:
def market_is_open():
    #get current time
    current_time = datetime.now()
    #format currrent time
    current_time_form = (current_time.hour * 100) + current_time.minute
    if current_time_form >= 600 and current_time_form <= 1230:
        return True
    else:
        return False

In [ ]:
def trade():
    #get current time
    current_time = datetime.now()
    #Inform user that process is beginning again
    print("Algorithm starting at ", current_time.hour, current_time.minute)
    
    #apply filters to tradeable stocks
    #push_stock_purchase_list_to_cloud(all_tradeable_stocks)
    filter_tradeable_stocks(all_tradeable_stocks)
    
    #get purchase list 
    #purchase_list = get_purchase_list_from_cloud()
    purchase_list = get_sorted_purchase_list()
    
    #if there is more than 0 stocks in purchase list
    if len(purchase_list) > 0:
        #send purchase list to buy stock which tries to purchase stocks
        buy_stock(purchase_list)

In [ ]:
def sell_order_past_hour(order_time_conv_pst):
    curr_time = datetime.now()
    curr_time_form = curr_time.strftime("%Y-%m-%d %H:%M:%S")
    curr_time_form = str(curr_time_form)
    
    curr_time_conv = datetime.strptime(curr_time_form, "%Y-%m-%d %H:%M:%S")

    diff = curr_time_conv-order_time_conv_pst
    
    if diff.total_seconds() > 3600:
        print(diff.total_seconds())
        return True
    else:
        return False
    

In [ ]:
def get_PST_time(order_time):
    order_time_form = order_time.replace('T', " ")
    order_time_form = order_time_form.replace ('Z', "")
    order_time_conv = datetime.strptime(order_time_form, "%Y-%m-%d %H:%M:%S.%f")
    order_time_conv_pst = order_time_conv - timedelta(hours=7) 
    return order_time_conv_pst

In [ ]:
def get_sell_limit_orders():
    sell_limit_orders = []
    #get all stock orders
    orders_list = rs.get_all_stock_orders()
    for orders in orders_list:
        if orders['side'] == 'sell' and orders['state'] == 'confirmed':
            #order_id and created_time
            #print(orders)
            #print('\n')
            #instrument needed
            instrument = orders['instrument']
            ticker = rs.stocks.get_symbol_by_url(instrument)
            print(ticker)
            
            append_val = [orders['id'], orders['created_at'], ticker]
            sell_limit_orders.append(append_val)
    return sell_limit_orders

In [ ]:
#DO THIS MANUALLY BEFORE PUTTING INTO PROD

#get orderIDs for all sell limit order
sell_limit_orders = get_sell_limit_orders()
#if sell limit order has been in place for more than an hour
for orders in sell_limit_orders:
    order_time_conv_pst = get_PST_time(orders[1])
    if sell_order_past_hour(order_time_conv_pst):
        print ("past hour")
        
        #cancel the sell limit order
        #rs.orders.cancel_stock_order(orders[0])
        
        #PROBABLY HAVE TO CHECK THAT ORDER IS CANCELLED BEFORE SELLING SHARE
        
        #place an immediate sell order for that share
        #robin_stocks.orders.order_sell_market(symbol, shares, timeInForce='gtc', extendedHours=False)
        #rs.orders.order_sell_market(orders[2], 1, timeInForce='gtc', extendedHours=False)

In [ ]:
all_tradeable_stocks = get_tradeable_stocks()

In [ ]:
while True:
    if market_is_open():
        trade()